Парсер кратких описаний постов и комментариев к ним с https://varlamov.ru/<br>Позволяет собрать данные за определенный год.<br>Текущее ограничение - собирает только верхние комментарии в ветках.<br>Работает с использованием библиотеки scrapy

In [1]:
import pandas as pd
import re
import urllib.request

from scrapy.selector import Selector
from scrapy.utils.markup import remove_tags
from scrapy.http import HtmlResponse
from tqdm import tqdm

from datetime import datetime
import numpy as np

In [2]:
# Функция генерирует ссылки на страницы блога varlrmov с постами по дням
def gen_daily_urls(blog_name, year):
    calendar = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    calendar_leap = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    urls = []
    for month in range(12):
        if year in [2018]: #TODO current year
            if (month < (datetime.now().month-1) and month < 9):
                for day in range(1, calendar_leap[month] + 1):
                    if day < 10:
                        urls.append('https://' + blog_name + '/' + str(year) + '/0' + str(month + 1) +
                                    '/0' + str(day) + '/')
                    else:
                        urls.append('https://' + blog_name + '/' + str(year) + '/0' + str(month + 1) +
                                    '/' + str(day) + '/')
            elif (month < (datetime.now().month-1) and month >= 9):
                for day in range(1, calendar_leap[month] + 1):
                    if day < 10:
                        urls.append('https://' + blog_name + '/' + str(year) + '/' + str(month + 1) +
                                    '/0' + str(day) + '/')
                    else:
                        urls.append('https://' + blog_name + '/' + str(year) + '/' + str(month + 1) +
                                    '/' + str(day) + '/')
            if (month == (datetime.now().month-1) and month < 9):
                for day in range(1, calendar_leap[month] + 1):
                    if (day <= datetime.now().day and day < 10):
                        urls.append('https://' + blog_name + '/' + str(year) + '/0' + str(month + 1) +
                                    '/0' + str(day) + '/')
                    elif (day <= datetime.now().day and day >= 10):
                        urls.append('https://' + blog_name + '/' + str(year) + '/0' + str(month + 1) +
                                    '/' + str(day) + '/')
            elif (month == (datetime.now().month-1) and month >= 9):
                for day in range(1, calendar_leap[month] + 1):
                    if (day <= datetime.now().day and day < 10):
                        urls.append('https://' + blog_name + '/' + str(year) + '/' + str(month + 1) +
                                    '/0' + str(day) + '/')
                    elif (day <= datetime.now().day and day >= 10):
                        urls.append('https://' + blog_name + '/' + str(year) + '/' + str(month + 1) +
                                    '/' + str(day) + '/')
        else:
            if month < 9:
                for day in range(1, calendar_leap[month] + 1):
                    if day < 10:
                        urls.append('https://' + blog_name + '/' + str(year) + '/0' + str(month + 1) +
                                    '/0' + str(day) + '/')
                    else:
                        urls.append('https://' + blog_name + '/' + str(year) + '/0' + str(month + 1) +
                                    '/' + str(day) + '/')
            else:
                for day in range(1, calendar_leap[month] + 1):
                    if day < 10:
                        urls.append('https://' + blog_name + '/' + str(year) + '/' + str(month + 1) +
                                    '/0' + str(day) + '/')
                    else:
                        urls.append('https://' + blog_name + '/' + str(year) + '/' + str(month + 1) +
                                    '/' + str(day) + '/')
    return urls

In [3]:
# функция собирает информацию о статьях в словарь
def get_articles(daily_urls):
    d = {'URL': [], 'Publish Date': [], 'Title': [], 'Lead sentences': []}
    for url in daily_urls:
        try:
            req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
            html = urllib.request.urlopen(url)
            html = html.read()
            
            link = Selector(text=html).xpath("//*[contains(@class, 'j-e-title')]/a/@href").extract()
            d['URL'].extend(link)

            pubdate = []
            pubdate.append(datetime.strptime(url[-11:-1], '%Y/%m/%d'))
            d['Publish Date'].extend(pubdate * len(link))
            
            title = Selector(text=html).xpath("//*[contains(@class, 'j-e-title')]/a/text()").extract()
            d['Title'].extend(title)
            
            sel = Selector(text=html).xpath("//*[contains(@class, 'j-e-text')]")
            lead = []
            for node in sel:
                lead.append(''.join(node.xpath("text()").extract()).strip())
            d['Lead sentences'].extend(lead)
        except:
            pass
    return d

In [4]:
# функция по листу с url собирает комментарии к статьям
def get_comments(urls):
    d = {'URL': [], 'comment': []}
    i = 1
    for url in urls:
        try:        
            req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
            html = urllib.request.urlopen(url)
            html = html.read()
            comm = remove_tags(str(Selector(text=html).xpath("//*[contains(@class, 'j-c-text') and contains(@itemprop, 'commentText') and contains(@itemprop, 'name')]").extract()))
            comm = re.sub('Edited at [0-9- :]{17}[(UTC)]{5}', '', comm)
            comm = comm.replace('\\xa0', ' ')
            comm = comm[2:-2].split(sep="\', \'")
            d['URL'].extend([url] * len(comm))
            d['comment'].extend(comm)
            print('{:.2f}%, successful scrapping for {}'.format(i*100/len(urls), url))
            i += 1
        except:
            print('{:.2f}%, unsuccessful scrapping for {}'.format(i*100/len(urls), url))
            i += 1
            pass
    return d

In [5]:
def ljscrapper(blog_name, year):
    # скраплю краткую информацию о статьях, пишу в csv:
    daily_urls = gen_daily_urls(blog_name, year)
    articles = get_articles(daily_urls)
    print("Собрана информация о статьях за : ", year, " год:", {key: len(value) for key, value in articles.items()})
    df = pd.DataFrame(data = articles)
    df.to_csv('./csv/' + blog_name + '_' + str(year) +'_articles.csv', sep='|', index=False, encoding='utf-8')
    # скраплю комментарии, пишу в csv:
    comments = get_comments(articles['URL'])
    print('Собрано комментариев за', year, 'год: ', {key: len(value) for key, value in comments.items()})
    df = pd.DataFrame(data = comments)
    df.to_csv('./csv/' + blog_name + '_' + str(year) +'_comments.csv', sep='|', index=False, encoding='utf-8')    
    return None

In [7]:
%%time
ljscrapper('varlamov.ru', 2018)

Собрана информация о статьях за :  2018  год: {'URL': 326, 'Publish Date': 326, 'Title': 326, 'Lead sentences': 326}
0.31%, successful scrapping for https://varlamov.ru/2729646.html
0.61%, successful scrapping for https://varlamov.ru/2729833.html
0.92%, successful scrapping for https://varlamov.ru/2730036.html
1.23%, successful scrapping for https://varlamov.ru/2730315.html
1.53%, successful scrapping for https://varlamov.ru/2730676.html
1.84%, successful scrapping for https://varlamov.ru/2730802.html
2.15%, successful scrapping for https://varlamov.ru/2731154.html
2.45%, successful scrapping for https://varlamov.ru/2731382.html
2.76%, successful scrapping for https://varlamov.ru/2731651.html
3.07%, successful scrapping for https://varlamov.ru/2732189.html
3.37%, successful scrapping for https://varlamov.ru/2732464.html
3.68%, successful scrapping for https://varlamov.ru/2731820.html
3.99%, successful scrapping for https://varlamov.ru/2732648.html
4.29%, successful scrapping for https:

75.77%, successful scrapping for https://varlamov.ru/2791855.html
76.07%, successful scrapping for https://varlamov.ru/2791954.html
76.38%, successful scrapping for https://varlamov.ru/2792262.html
76.69%, successful scrapping for https://varlamov.ru/2792457.html
76.99%, successful scrapping for https://varlamov.ru/2792826.html
77.30%, successful scrapping for https://varlamov.ru/2793062.html
77.61%, successful scrapping for https://varlamov.ru/2793348.html
77.91%, successful scrapping for https://varlamov.ru/2793507.html
78.22%, successful scrapping for https://varlamov.ru/2793825.html
78.53%, successful scrapping for https://varlamov.ru/2789075.html
78.83%, successful scrapping for https://varlamov.ru/2794169.html
79.14%, successful scrapping for https://varlamov.ru/2794637.html
79.45%, successful scrapping for https://varlamov.ru/2794907.html
79.75%, successful scrapping for https://varlamov.ru/2795061.html
80.06%, successful scrapping for https://varlamov.ru/2795363.html
80.37%, su

In [8]:
%%time
ljscrapper('varlamov.ru', 2017)

Собрана информация о статьях за :  2017  год: {'URL': 2198, 'Publish Date': 2198, 'Title': 2198, 'Lead sentences': 2198}
0.05%, successful scrapping for https://varlamov.ru/2166097.html
0.09%, successful scrapping for https://varlamov.ru/2166365.html
0.14%, successful scrapping for https://varlamov.ru/2166646.html
0.18%, successful scrapping for https://varlamov.ru/2165891.html
0.23%, successful scrapping for https://varlamov.ru/2167116.html
0.27%, successful scrapping for https://varlamov.ru/2167360.html
0.32%, successful scrapping for https://varlamov.ru/2167759.html
0.36%, successful scrapping for https://varlamov.ru/2167848.html
0.41%, successful scrapping for https://varlamov.ru/2168102.html
0.45%, successful scrapping for https://varlamov.ru/2168417.html
0.50%, successful scrapping for https://varlamov.ru/2168736.html
0.55%, successful scrapping for https://varlamov.ru/2168932.html
0.59%, successful scrapping for https://varlamov.ru/2169163.html
0.64%, successful scrapping for ht

11.37%, successful scrapping for https://varlamov.ru/2229095.html
11.42%, successful scrapping for https://varlamov.ru/2228563.html
11.46%, successful scrapping for https://varlamov.ru/2229661.html
11.51%, successful scrapping for https://varlamov.ru/2229814.html
11.56%, successful scrapping for https://varlamov.ru/2230074.html
11.60%, successful scrapping for https://varlamov.ru/2230498.html
11.65%, successful scrapping for https://varlamov.ru/2230770.html
11.69%, successful scrapping for https://varlamov.ru/2231024.html
11.74%, successful scrapping for https://varlamov.ru/2231091.html
11.78%, successful scrapping for https://varlamov.ru/2231420.html
11.83%, successful scrapping for https://varlamov.ru/2231725.html
11.87%, successful scrapping for https://varlamov.ru/2231844.html
11.92%, successful scrapping for https://varlamov.ru/2232192.html
11.97%, successful scrapping for https://varlamov.ru/2232672.html
12.01%, successful scrapping for https://varlamov.ru/2232473.html
12.06%, su

22.66%, successful scrapping for https://varlamov.ru/2291191.html
22.70%, successful scrapping for https://varlamov.ru/2291405.html
22.75%, successful scrapping for https://varlamov.ru/2291502.html
22.79%, successful scrapping for https://varlamov.ru/2278438.html
22.84%, successful scrapping for https://varlamov.ru/2291777.html
22.88%, successful scrapping for https://varlamov.ru/2292101.html
22.93%, successful scrapping for https://varlamov.ru/2007898.html
22.98%, successful scrapping for https://varlamov.ru/2292331.html
23.02%, successful scrapping for https://varlamov.ru/2292619.html
23.07%, successful scrapping for https://varlamov.ru/2292777.html
23.11%, successful scrapping for https://varlamov.ru/2293141.html
23.16%, successful scrapping for https://varlamov.ru/2293498.html
23.20%, successful scrapping for https://varlamov.ru/2293543.html
23.25%, successful scrapping for https://varlamov.ru/2293984.html
23.29%, successful scrapping for https://varlamov.ru/2294410.html
23.34%, su

33.94%, successful scrapping for https://varlamov.ru/2355472.html
33.99%, successful scrapping for https://varlamov.ru/2355945.html
34.03%, successful scrapping for https://varlamov.ru/2349518.html
34.08%, successful scrapping for https://varlamov.ru/2356115.html
34.12%, successful scrapping for https://varlamov.ru/2356579.html
34.17%, successful scrapping for https://varlamov.ru/2356824.html
34.21%, successful scrapping for https://varlamov.ru/2356436.html
34.26%, successful scrapping for https://varlamov.ru/2357058.html
34.30%, successful scrapping for https://varlamov.ru/2357665.html
34.35%, successful scrapping for https://varlamov.ru/2357285.html
34.39%, successful scrapping for https://varlamov.ru/2357796.html
34.44%, successful scrapping for https://varlamov.ru/2358156.html
34.49%, successful scrapping for https://varlamov.ru/2358314.html
34.53%, successful scrapping for https://varlamov.ru/2358654.html
34.58%, successful scrapping for https://varlamov.ru/2359127.html
34.62%, su

45.22%, successful scrapping for https://varlamov.ru/2419808.html
45.27%, successful scrapping for https://varlamov.ru/2420465.html
45.31%, successful scrapping for https://varlamov.ru/2420765.html
45.36%, successful scrapping for https://varlamov.ru/2421155.html
45.40%, successful scrapping for https://varlamov.ru/2420516.html
45.45%, successful scrapping for https://varlamov.ru/2421392.html
45.50%, successful scrapping for https://varlamov.ru/2421758.html
45.54%, successful scrapping for https://varlamov.ru/2419969.html
45.59%, successful scrapping for https://varlamov.ru/2421854.html
45.63%, successful scrapping for https://varlamov.ru/2422030.html
45.68%, successful scrapping for https://varlamov.ru/2422466.html
45.72%, successful scrapping for https://varlamov.ru/2422700.html
45.77%, successful scrapping for https://varlamov.ru/2422965.html
45.81%, successful scrapping for https://varlamov.ru/2423127.html
45.86%, successful scrapping for https://varlamov.ru/2423555.html
45.91%, su

56.51%, successful scrapping for https://varlamov.ru/2484990.html
56.55%, successful scrapping for https://varlamov.ru/2485227.html
56.60%, successful scrapping for https://varlamov.ru/2485392.html
56.64%, successful scrapping for https://varlamov.ru/2485538.html
56.69%, successful scrapping for https://varlamov.ru/2486204.html
56.73%, successful scrapping for https://varlamov.ru/2486776.html
56.78%, successful scrapping for https://varlamov.ru/2487022.html
56.82%, successful scrapping for https://varlamov.ru/2485823.html
56.87%, successful scrapping for https://varlamov.ru/2487284.html
56.92%, successful scrapping for https://varlamov.ru/2482460.html
56.96%, successful scrapping for https://varlamov.ru/2487351.html
57.01%, successful scrapping for https://varlamov.ru/2487628.html
57.05%, successful scrapping for https://varlamov.ru/2488034.html
57.10%, successful scrapping for https://varlamov.ru/2488116.html
57.14%, successful scrapping for https://varlamov.ru/2488643.html
57.19%, su

67.79%, successful scrapping for https://varlamov.ru/2548145.html
67.83%, successful scrapping for https://varlamov.ru/2548358.html
67.88%, successful scrapping for https://varlamov.ru/2548565.html
67.93%, successful scrapping for https://varlamov.ru/2549123.html
67.97%, successful scrapping for https://varlamov.ru/2549367.html
68.02%, successful scrapping for https://varlamov.ru/2549827.html
68.06%, successful scrapping for https://varlamov.ru/2550205.html
68.11%, successful scrapping for https://varlamov.ru/2550507.html
68.15%, successful scrapping for https://varlamov.ru/2550770.html
68.20%, successful scrapping for https://varlamov.ru/2550831.html
68.24%, successful scrapping for https://varlamov.ru/2551352.html
68.29%, successful scrapping for https://varlamov.ru/2551993.html
68.33%, successful scrapping for https://varlamov.ru/2551661.html
68.38%, successful scrapping for https://varlamov.ru/2552222.html
68.43%, successful scrapping for https://varlamov.ru/2552425.html
68.47%, su

79.07%, successful scrapping for https://varlamov.ru/2611267.html
79.12%, successful scrapping for https://varlamov.ru/2611624.html
79.16%, successful scrapping for https://varlamov.ru/2611876.html
79.21%, successful scrapping for https://varlamov.ru/2612052.html
79.25%, successful scrapping for https://varlamov.ru/2612463.html
79.30%, successful scrapping for https://varlamov.ru/2612564.html
79.34%, successful scrapping for https://varlamov.ru/2612796.html
79.39%, successful scrapping for https://varlamov.ru/2613053.html
79.44%, successful scrapping for https://varlamov.ru/2613272.html
79.48%, successful scrapping for https://varlamov.ru/2613613.html
79.53%, successful scrapping for https://varlamov.ru/2613842.html
79.57%, successful scrapping for https://varlamov.ru/2614056.html
79.62%, successful scrapping for https://varlamov.ru/2614377.html
79.66%, successful scrapping for https://varlamov.ru/2614695.html
79.71%, successful scrapping for https://varlamov.ru/2614964.html
79.75%, su

90.35%, successful scrapping for https://varlamov.ru/2674158.html
90.40%, successful scrapping for https://varlamov.ru/2674209.html
90.45%, successful scrapping for https://varlamov.ru/2674911.html
90.49%, successful scrapping for https://varlamov.ru/2674974.html
90.54%, successful scrapping for https://varlamov.ru/2675315.html
90.58%, successful scrapping for https://varlamov.ru/2675541.html
90.63%, successful scrapping for https://varlamov.ru/2675756.html
90.67%, successful scrapping for https://varlamov.ru/2676084.html
90.72%, successful scrapping for https://varlamov.ru/2676278.html
90.76%, successful scrapping for https://varlamov.ru/2676662.html
90.81%, successful scrapping for https://varlamov.ru/2676986.html
90.86%, successful scrapping for https://varlamov.ru/2677126.html
90.90%, successful scrapping for https://varlamov.ru/2677294.html
90.95%, successful scrapping for https://varlamov.ru/2677679.html
90.99%, successful scrapping for https://varlamov.ru/2677889.html
91.04%, su

In [6]:
%%time
ljscrapper('varlamov.ru', 2016)

Собрана информация о статьях за :  2016  год: {'URL': 2317, 'Publish Date': 2317, 'Title': 2317, 'Lead sentences': 2317}
0.04%, successful scrapping for https://varlamov.ru/1557244.html
0.09%, successful scrapping for https://varlamov.ru/2164929.html
0.13%, successful scrapping for https://varlamov.ru/1557291.html
0.17%, successful scrapping for https://varlamov.ru/1557558.html
0.22%, successful scrapping for https://varlamov.ru/1558056.html
0.26%, successful scrapping for https://varlamov.ru/1558388.html
0.30%, successful scrapping for https://varlamov.ru/1558762.html
0.35%, successful scrapping for https://varlamov.ru/1558952.html
0.39%, successful scrapping for https://varlamov.ru/1996937.html
0.43%, successful scrapping for https://varlamov.ru/1559175.html
0.47%, successful scrapping for https://varlamov.ru/1559335.html
0.52%, successful scrapping for https://varlamov.ru/1559731.html
0.56%, successful scrapping for https://varlamov.ru/1559987.html
0.60%, successful scrapping for ht

10.79%, successful scrapping for https://varlamov.ru/1620628.html
10.83%, successful scrapping for https://varlamov.ru/1620815.html
10.88%, successful scrapping for https://varlamov.ru/1621097.html
10.92%, successful scrapping for https://varlamov.ru/1621419.html
10.96%, successful scrapping for https://varlamov.ru/1621613.html
11.01%, successful scrapping for https://varlamov.ru/1621782.html
11.05%, successful scrapping for https://varlamov.ru/1622139.html
11.09%, successful scrapping for https://varlamov.ru/1622371.html
11.14%, successful scrapping for https://varlamov.ru/1622556.html
11.18%, successful scrapping for https://varlamov.ru/1622822.html
11.22%, successful scrapping for https://varlamov.ru/1623178.html
11.26%, successful scrapping for https://varlamov.ru/1623469.html
11.31%, successful scrapping for https://varlamov.ru/1623722.html
11.35%, successful scrapping for https://varlamov.ru/1624053.html
11.39%, successful scrapping for https://varlamov.ru/1624104.html
11.44%, su

21.49%, successful scrapping for https://varlamov.ru/1690215.html
21.54%, successful scrapping for https://varlamov.ru/1690532.html
21.58%, successful scrapping for https://varlamov.ru/1690780.html
21.62%, successful scrapping for https://varlamov.ru/1691000.html
21.67%, successful scrapping for https://varlamov.ru/1691271.html
21.71%, successful scrapping for https://varlamov.ru/1691479.html
21.75%, successful scrapping for https://varlamov.ru/1691787.html
21.80%, successful scrapping for https://varlamov.ru/1692415.html
21.84%, successful scrapping for https://varlamov.ru/1691964.html
21.88%, successful scrapping for https://varlamov.ru/1692678.html
21.92%, successful scrapping for https://varlamov.ru/1693018.html
21.97%, successful scrapping for https://varlamov.ru/1693567.html
22.01%, successful scrapping for https://varlamov.ru/1693373.html
22.05%, successful scrapping for https://varlamov.ru/1693883.html
22.10%, successful scrapping for https://varlamov.ru/1694193.html
22.14%, su

32.20%, successful scrapping for https://varlamov.ru/1756160.html
32.24%, successful scrapping for https://varlamov.ru/1753704.html
32.28%, successful scrapping for https://varlamov.ru/1756632.html
32.33%, successful scrapping for https://varlamov.ru/1754313.html
32.37%, successful scrapping for https://varlamov.ru/1756762.html
32.41%, successful scrapping for https://varlamov.ru/1728438.html
32.46%, successful scrapping for https://varlamov.ru/1712997.html
32.50%, successful scrapping for https://varlamov.ru/1757133.html
32.54%, successful scrapping for https://varlamov.ru/1755036.html
32.59%, successful scrapping for https://varlamov.ru/1757335.html
32.63%, successful scrapping for https://varlamov.ru/1757453.html
32.67%, successful scrapping for https://varlamov.ru/1758047.html
32.71%, successful scrapping for https://varlamov.ru/1758409.html
32.76%, successful scrapping for https://varlamov.ru/1758713.html
32.80%, successful scrapping for https://varlamov.ru/1758908.html
32.84%, su

42.90%, successful scrapping for https://varlamov.ru/1818731.html
42.94%, successful scrapping for https://varlamov.ru/1819270.html
42.99%, successful scrapping for https://varlamov.ru/1819037.html
43.03%, successful scrapping for https://varlamov.ru/1819531.html
43.07%, successful scrapping for https://varlamov.ru/1819884.html
43.12%, successful scrapping for https://varlamov.ru/1820025.html
43.16%, successful scrapping for https://varlamov.ru/1820189.html
43.20%, successful scrapping for https://varlamov.ru/1820623.html
43.25%, successful scrapping for https://varlamov.ru/1809542.html
43.29%, successful scrapping for https://varlamov.ru/1821265.html
43.33%, successful scrapping for https://varlamov.ru/1821693.html
43.38%, successful scrapping for https://varlamov.ru/1821857.html
43.42%, successful scrapping for https://varlamov.ru/1822265.html
43.46%, successful scrapping for https://varlamov.ru/1822535.html
43.50%, successful scrapping for https://varlamov.ru/1822848.html
43.55%, su

53.60%, successful scrapping for https://varlamov.ru/1882614.html
53.65%, successful scrapping for https://varlamov.ru/1882671.html
53.69%, successful scrapping for https://varlamov.ru/1882023.html
53.73%, successful scrapping for https://varlamov.ru/1857293.html
53.78%, successful scrapping for https://varlamov.ru/1883120.html
53.82%, successful scrapping for https://varlamov.ru/1883147.html
53.86%, successful scrapping for https://varlamov.ru/1883794.html
53.91%, successful scrapping for https://varlamov.ru/1869894.html
53.95%, successful scrapping for https://varlamov.ru/1884507.html
53.99%, successful scrapping for https://varlamov.ru/1884774.html
54.04%, successful scrapping for https://varlamov.ru/1885130.html
54.08%, successful scrapping for https://varlamov.ru/1884325.html
54.12%, successful scrapping for https://varlamov.ru/1885625.html
54.16%, successful scrapping for https://varlamov.ru/1885366.html
54.21%, successful scrapping for https://varlamov.ru/1886068.html
54.25%, su

64.31%, successful scrapping for https://varlamov.ru/1949973.html
64.35%, successful scrapping for https://varlamov.ru/1950443.html
64.39%, successful scrapping for https://varlamov.ru/1950676.html
64.44%, successful scrapping for https://varlamov.ru/1951004.html
64.48%, successful scrapping for https://varlamov.ru/1951613.html
64.52%, successful scrapping for https://varlamov.ru/1951902.html
64.57%, successful scrapping for https://varlamov.ru/1952366.html
64.61%, successful scrapping for https://varlamov.ru/1952638.html
64.65%, successful scrapping for https://varlamov.ru/1952953.html
64.70%, successful scrapping for https://varlamov.ru/1953171.html
64.74%, successful scrapping for https://varlamov.ru/1953538.html
64.78%, successful scrapping for https://varlamov.ru/1930822.html
64.83%, successful scrapping for https://varlamov.ru/1953888.html
64.87%, successful scrapping for https://varlamov.ru/1953391.html
64.91%, successful scrapping for https://varlamov.ru/1954213.html
64.95%, su

74.97%, successful scrapping for https://varlamov.ru/2014676.html
75.01%, successful scrapping for https://varlamov.ru/2014726.html
75.05%, successful scrapping for https://varlamov.ru/2015026.html
75.10%, successful scrapping for https://varlamov.ru/2015440.html
75.14%, successful scrapping for https://varlamov.ru/2015620.html
75.18%, successful scrapping for https://varlamov.ru/2015860.html
75.23%, successful scrapping for https://varlamov.ru/1995424.html
75.27%, successful scrapping for https://varlamov.ru/2016034.html
75.31%, successful scrapping for https://varlamov.ru/2016345.html
75.36%, successful scrapping for https://varlamov.ru/2016692.html
75.40%, successful scrapping for https://varlamov.ru/2016868.html
75.44%, successful scrapping for https://varlamov.ru/2017046.html
75.49%, successful scrapping for https://varlamov.ru/2017321.html
75.53%, successful scrapping for https://varlamov.ru/2017613.html
75.57%, successful scrapping for https://varlamov.ru/2017921.html
75.62%, su

85.67%, successful scrapping for https://varlamov.ru/2037612.html
85.71%, successful scrapping for https://varlamov.ru/2079629.html
85.76%, successful scrapping for https://varlamov.ru/2080139.html
85.80%, successful scrapping for https://varlamov.ru/2080480.html
85.84%, successful scrapping for https://varlamov.ru/2080625.html
85.89%, successful scrapping for https://varlamov.ru/2080868.html
85.93%, successful scrapping for https://varlamov.ru/2075009.html
85.97%, successful scrapping for https://varlamov.ru/2081071.html
86.02%, successful scrapping for https://varlamov.ru/2081744.html
86.06%, successful scrapping for https://varlamov.ru/2082001.html
86.10%, successful scrapping for https://varlamov.ru/2082292.html
86.15%, successful scrapping for https://varlamov.ru/2082453.html
86.19%, successful scrapping for https://varlamov.ru/2081519.html
86.23%, successful scrapping for https://varlamov.ru/2077863.html
86.28%, successful scrapping for https://varlamov.ru/2082585.html
86.32%, su

96.37%, successful scrapping for https://varlamov.ru/2141441.html
96.42%, successful scrapping for https://varlamov.ru/2141840.html
96.46%, successful scrapping for https://varlamov.ru/2142406.html
96.50%, successful scrapping for https://varlamov.ru/2142678.html
96.55%, successful scrapping for https://varlamov.ru/2142860.html
96.59%, successful scrapping for https://varlamov.ru/2142987.html
96.63%, successful scrapping for https://varlamov.ru/2143462.html
96.68%, successful scrapping for https://varlamov.ru/2143980.html
96.72%, successful scrapping for https://varlamov.ru/2144400.html
96.76%, successful scrapping for https://varlamov.ru/2144554.html
96.81%, successful scrapping for https://varlamov.ru/2143710.html
96.85%, successful scrapping for https://varlamov.ru/2144786.html
96.89%, successful scrapping for https://varlamov.ru/2145076.html
96.94%, successful scrapping for https://varlamov.ru/2145992.html
96.98%, successful scrapping for https://varlamov.ru/2146240.html
97.02%, su

In [7]:
%%time
ljscrapper('varlamov.ru', 2015)

Собрана информация о статьях за :  2015  год: {'URL': 1186, 'Publish Date': 1186, 'Title': 1186, 'Lead sentences': 1186}
0.08%, successful scrapping for https://varlamov.ru/1243402.html
0.17%, successful scrapping for https://varlamov.ru/1243792.html
0.25%, successful scrapping for https://varlamov.ru/1244012.html
0.34%, successful scrapping for https://varlamov.ru/1244248.html
0.42%, successful scrapping for https://varlamov.ru/1244437.html
0.51%, successful scrapping for https://varlamov.ru/1244780.html
0.59%, successful scrapping for https://varlamov.ru/1245062.html
0.67%, successful scrapping for https://varlamov.ru/1245339.html
0.76%, successful scrapping for https://varlamov.ru/1245448.html
0.84%, successful scrapping for https://varlamov.ru/1245776.html
0.93%, successful scrapping for https://varlamov.ru/1245985.html
1.01%, successful scrapping for https://varlamov.ru/1246607.html
1.10%, successful scrapping for https://varlamov.ru/1246923.html
1.18%, successful scrapping for ht

20.99%, successful scrapping for https://varlamov.ru/1309833.html
21.08%, successful scrapping for https://varlamov.ru/1310112.html
21.16%, successful scrapping for https://varlamov.ru/1310442.html
21.25%, successful scrapping for https://varlamov.ru/1310654.html
21.33%, successful scrapping for https://varlamov.ru/1310723.html
21.42%, successful scrapping for https://varlamov.ru/1311046.html
21.50%, successful scrapping for https://varlamov.ru/1311289.html
21.59%, successful scrapping for https://varlamov.ru/1311534.html
21.67%, successful scrapping for https://varlamov.ru/1311966.html
21.75%, successful scrapping for https://varlamov.ru/1312122.html
21.84%, successful scrapping for https://varlamov.ru/1312297.html
21.92%, successful scrapping for https://varlamov.ru/1312660.html
22.01%, successful scrapping for https://varlamov.ru/1312982.html
22.09%, successful scrapping for https://varlamov.ru/1313220.html
22.18%, successful scrapping for https://varlamov.ru/1313283.html
22.26%, su

41.82%, successful scrapping for https://varlamov.ru/1375399.html
41.91%, successful scrapping for https://varlamov.ru/1375650.html
41.99%, successful scrapping for https://varlamov.ru/1375973.html
42.07%, successful scrapping for https://varlamov.ru/1376481.html
42.16%, successful scrapping for https://varlamov.ru/1376749.html
42.24%, successful scrapping for https://varlamov.ru/1376994.html
42.33%, successful scrapping for https://varlamov.ru/1377163.html
42.41%, successful scrapping for https://varlamov.ru/1377280.html
42.50%, successful scrapping for https://varlamov.ru/1377584.html
42.58%, successful scrapping for https://varlamov.ru/1377821.html
42.66%, successful scrapping for https://varlamov.ru/1378181.html
42.75%, successful scrapping for https://varlamov.ru/1378527.html
42.83%, successful scrapping for https://varlamov.ru/1378788.html
42.92%, successful scrapping for https://varlamov.ru/1378953.html
43.00%, successful scrapping for https://varlamov.ru/1379236.html
43.09%, su

62.73%, successful scrapping for https://varlamov.ru/1439788.html
62.82%, successful scrapping for https://varlamov.ru/1440006.html
62.90%, successful scrapping for https://varlamov.ru/1440434.html
62.98%, successful scrapping for https://varlamov.ru/1440544.html
63.07%, successful scrapping for https://varlamov.ru/1440968.html
63.15%, successful scrapping for https://varlamov.ru/1441168.html
63.24%, successful scrapping for https://varlamov.ru/1441314.html
63.32%, successful scrapping for https://varlamov.ru/1441552.html
63.41%, successful scrapping for https://varlamov.ru/1442016.html
63.49%, successful scrapping for https://varlamov.ru/1442281.html
63.58%, successful scrapping for https://varlamov.ru/1442556.html
63.66%, successful scrapping for https://varlamov.ru/1442610.html
63.74%, successful scrapping for https://varlamov.ru/1442982.html
63.83%, successful scrapping for https://varlamov.ru/1443297.html
63.91%, successful scrapping for https://varlamov.ru/1443396.html
64.00%, su

83.64%, successful scrapping for https://varlamov.ru/1505023.html
83.73%, successful scrapping for https://varlamov.ru/1505171.html
83.81%, successful scrapping for https://varlamov.ru/1505437.html
83.90%, successful scrapping for https://varlamov.ru/1505618.html
83.98%, successful scrapping for https://varlamov.ru/1505874.html
84.06%, successful scrapping for https://varlamov.ru/1506277.html
84.15%, successful scrapping for https://varlamov.ru/1506425.html
84.23%, successful scrapping for https://varlamov.ru/1506625.html
84.32%, successful scrapping for https://varlamov.ru/1506821.html
84.40%, successful scrapping for https://varlamov.ru/1507107.html
84.49%, successful scrapping for https://varlamov.ru/1507455.html
84.57%, successful scrapping for https://varlamov.ru/1507584.html
84.65%, successful scrapping for https://varlamov.ru/1507923.html
84.74%, successful scrapping for https://varlamov.ru/1508225.html
84.82%, successful scrapping for https://varlamov.ru/1508513.html
84.91%, su